# 03 - Coupled Oscillators

**Overview**

This notebook guides you through the simulation of multiple harmonic systems coupled together. The notebook will analyze a molecule composed by three atoms linked by two bonds. As the mass of the central atom is varied, the two bonds will become more or less coupled, giving rise to a motion that seems to flow from one bond to the other. However, when analyzed using linear algebra (eingenvalues and eigenvectors), the motion reverts to the dynamics of two independent harmonic oscillators (normal modes).

In [ ]:
# @title Modules Setup { display-mode: "form" }
import numpy as np
# Install Plotly (if not already)
!pip install -q plotly > /dev/null
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm
!pip install -q rdkit > /dev/null
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw

**Problem**

Let's assume that we want to model the motion of small rigid (and highly symmetric) molecules, such as carbon dioxide (CO2), triiodide (I3-), cianide (HCN). At low temperature the energy associated with the bonds in these molecules has very deep minima and it is reasonable to assume the motion of each bond is going to be harmonic. However, when we consider both bonds at the same time, the motions of the individual atoms may become less easy to interpret. We want to understand how the relative masses of the atoms involved in the bonds affect the localized or delocalized nature of the molecular vibrations.

**Model**

We are going to assume that the atoms only move in the axial direction (1D problem) and we are going to model each bond using a classical spring. We will integrate the equations of motion using Velocity Verlet with a small timestep. By looking at systems with different masses, symmetric and asymmetric, we aim to understand how well the motions of the individual springs are preserved when they are combined together.

> Is the coupling between springs always going to be important? Can we still treat systems of coupled harmonic oscillators as a collection of independent vibrations?

To answer some of these questions, we will consider an ever simpler model, in which the central atom of the molecule is kept fixed by assigning it an infinite mass.

**Questions**

Before you run any simulation, answer the following question(s):

1. Considering you have a system of 3 atoms moving in 1 dimension, how many vibrations do you expect to see?
2. If the central atom is fixed, do you expect that the vibrations of the two bonds will affect each other? Considering a simplified problem where all the relevant quantities have unit values, what would be the frequencies of vibration of the two bonds?
3. In a problem where two atoms have masses significantly larger than the third (such as in HCN), do you think that the motion of the two bonds will be more or less affected by the fact they are connected through the central atom?   

Run the simulation, change the parameters, and run the simulation again as many times as needed to answer the following question(s):

4. Start with a system where the central atom is fixed. Consider both symmetric and asymmetric molecules (i.e. masses of the side atoms). Are the motions of the atoms coupled? Is there any potential energy moving from one side of the molecule to the other?
5. Now consider a symmetric system, but progressively reduce the mass of the central atom. When the three masses are identical you have a good model for I3-. How are the vibrations of the molecule, in particular their frequencies, changing? Would you say that reducing the mass makes the system more or less coupled?
6. Can you find any relation between the diagonal part of the dynamic matrix and the final frequencies of the vibrational modes?
7. Now consider an asymmetric system, where one of the atoms has a significantly smaller mass than the other two. Do you think this system is more or less 'coupled' than the symmetric system (i.e. the final trajectories and frequencies look more or less similar to the ones of two indipendend harmonic oscillators)?

In [ ]:
# @title Utility Functions { display-mode: "form" }
# --- functions to compute forces and energies ---
def compute_kinetic(velocities, masses):
    return 0.5 * np.sum(masses * velocities**2)

def compute_potential(positions, springs):
    U = 0.0
    for (i, j, props) in springs:
        k = props["k"]
        L0 = props["L0"]
        # displacement
        dx = positions[i] - positions[j]
        dist = abs(dx)
        # potential energy
        U += 0.5 * k * (dist - L0)**2
    return U

def compute_forces(nparticles, positions, springs):
    forces = np.zeros(nparticles)
    for (i, j, props) in springs:
        k = props["k"]
        L0 = props["L0"]
        # displacement
        dx = positions[i] - positions[j]
        dist = abs(dx)
        # avoid division by zero
        if dist == 0:
            continue
        # Hooke’s law
        F = -k * (dist - L0) * (dx / dist)
        # apply equal and opposite forces
        forces[i] += F
        forces[j] -= F
    return forces

def compute_hessian(nparticles, springs):
    H = np.zeros((nparticles, nparticles))
    for (i, j, props) in springs:
        k = props["k"]
        L0 = props["L0"]
        # spring contribution to Hessian
        H[i, i] += k
        H[j, j] += k
        H[i, j] -= k
        H[j, i] -= k
    return H

def compute_dynamic_matrix(nparticles, masses, springs):
    H = compute_hessian(nparticles, springs)
    D = np.zeros((nparticles, nparticles))
    for i in range(nparticles):
        for j in range(nparticles):
            D[i, j] = H[i, j] / np.sqrt(masses[i] * masses[j])
    return D

In [ ]:
# @title System Setup  { display-mode: "form" }
AB_eq_bond_length = 1.0
AC_eq_bond_length = 1.0
AB_bond_strenght = 1.0 # @param {type:"number"}
AC_bond_strenght = 1.0 # @param {type:"number"}
ABC_init_angle = 104 #convert to radians
ABC_angle_k = 100 #find optimal value here
A_mass = "infinite" # @param ["unit", "double", "large", "very large", "infinite"]
B_mass = 1  # @param {type:"number"}
C_mass = 1  # @param {type:"number"}

In [ ]:
# @title Static Analysis of the System  { display-mode: "form" }

mass_map = {
    "unit": 1.0,
    "double": 2.0,
    "large": 10.0,
    "very large": 20.0,
    "infinite": 1e10   # effectively pinned
}

# Initial conditions
A_pos = np.array(0.0,0.0)
B_pos = A_pos + np.array(AB_eq_bond_length*sin(ABC_init_angle/2),AB_eq_bond_length*cos(ABC_init_angle/2))
C_pos = A_pos - np.array(AC_eq_bond_length*sin(ABC_init_angle/2),AC_eq_bond_length*cos(ABC_init_angle/2))
# -- Define the system --
nparticles = 3
positions = np.array([B_pos, A_pos, C_pos])
masses = np.array([B_mass, mass_map[A_mass], C_mass])
nspring = 2
springs = [
    (0, 1, {"k": AB_bond_strenght, "L0": AB_eq_bond_length}),
    (1, 2, {"k": AC_bond_strenght, "L0": AC_eq_bond_length}),
    (0, 2, {"k": ABC_angle_k, "L0": ABC_bond_length}) #add angles here? maybe as a harmonic spring instead
]
# -- Compute the Dynamic Matrix of the system --
D = compute_dynamic_matrix(nparticles, masses, springs)

# -- Diagonalize the Dynamic Matrix --
eigenvalues, eigenvectors = np.linalg.eigh(D)

colors = ["red", "blue", "green"]

# Diagonal matrix from eigenvalues
Lambda = np.diag(eigenvalues)

# Equilibrium positions
eq_pos = positions

# Frequencies and energies
frequencies = np.sqrt(np.abs(eigenvalues))
energies = eigenvalues


In [ ]:
# @title Visualize Static Analysis of the System  { display-mode: "form" }
# --- Figure with a 2x2 grid ---
fig = plt.figure(figsize=(12, 8), constrained_layout=True)
gs = fig.add_gridspec(2, 2, width_ratios=[1, 1.5])  # left: matrices, right: modes

# --- Top-left: Dynamic matrix D ---
ax1 = fig.add_subplot(gs[0, 0])
norm = TwoSlopeNorm(vmin=min(D.min(), Lambda.min()) - 1e-1,
                    vcenter=0,
                    vmax=max(D.max(), Lambda.max()) + 1e-1)
cax1 = ax1.matshow(D, cmap="seismic", norm=norm)
ax1.set_title("Dynamic Matrix D")
for (i, j), val in np.ndenumerate(D):
    ax1.text(j, i, f"{val:.2f}", va="center", ha="center", color="white")

# --- Bottom-left: Diagonal matrix Λ ---
ax2 = fig.add_subplot(gs[1, 0])
cax2 = ax2.matshow(Lambda, cmap="seismic", norm=norm)
ax2.set_title("Diagonal Matrix (Eigenvalues)")
for (i, j), val in np.ndenumerate(Lambda):
    ax2.text(j, i, f"{val:.2f}", va="center", ha="center", color="white")

# Shared colorbar
fig.colorbar(cax1, ax=[ax1, ax2], shrink=0.8)

# --- Right column: 3 stacked normal modes ---
gs_right = gs[:, 1].subgridspec(nparticles, 1)

# compute global x range including arrows
#to-do: add y dimension here
all_x = []
for mode in range(nparticles):
    vec = eigenvectors[:, mode]
    scale = 0.3 * (eq_pos.max() - eq_pos.min())
    displacements = scale * vec / np.max(np.abs(vec))
    all_x.extend((eq_pos + displacements).tolist())

x_min = min(eq_pos.min(), min(all_x)) - 0.3
x_max = max(eq_pos.max(), max(all_x)) + 0.3

# vertical offsets for each particle (row separation)
y_offsets = np.linspace(0, 0.5, nparticles)  # e.g., [0, 0.5, 1]
y_offsets = y_offsets - np.mean(y_offsets)  # center around 0

for mode in range(nparticles):
    ax = fig.add_subplot(gs_right[mode])

    vec = eigenvectors[:, mode]
    scale = 0.3 * (eq_pos.max() - eq_pos.min())
    displacements = scale * vec / np.max(np.abs(vec))

    # plot equilibrium particle positions with vertical offsets
    ax.scatter(eq_pos, [0]*nparticles, s=200,
               c=colors, zorder=2)

    # plot arrows with same vertical offsets
    for i, x in enumerate(eq_pos):
        ax.arrow(x, y_offsets[i], displacements[i], 0,
                 head_width=0.05, head_length=0.05,
                 fc='k', ec='k', length_includes_head=True, zorder=3, color=colors[i])

    ax.set_title(f"Mode {mode+1}\nλ={eigenvalues[mode]:.3f}, ω={frequencies[mode]:.3f}")
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_offsets.min()-0.5, y_offsets.max()+0.5)  # enough room for all offsets
    ax.set_yticks([])
    if mode == nparticles - 1:
        ax.set_xlabel("Position")
    else:
        ax.set_xticklabels([])


plt.suptitle("Normal Mode Analysis", fontsize=16)
plt.show()

In [ ]:
# @title Classical Dynamics Parameters  { display-mode: "form" }
Temperature = 0.1 # @param {type:"number"}
dt = 0.01  # @param {type:"number"}
nsteps = 10000  # @param {type:"integer"}
frame_stride = 500 # @param {type:"integer"}
total_time = nsteps * dt
random_start = True # @param {type:"boolean"}
remove_com = True # @param {type:"boolean"}

In [ ]:
# @title Run the Simulation  { display-mode: "form" }
# Initial conditions
positions = np.array([B_pos, A_pos, C_pos]) #add y dimension here
# Initialize normal mode velocities from equipartition
kB = 1.0  # use 1 in reduced units
if random_start:
    qdot = np.random.normal(0, np.sqrt(kB * Temperature), size=nparticles)
else:
    qdot = np.ones(nparticles) * np.sqrt(kB * Temperature)
# Kill zero frequency modes
if remove_com:
    tol = 1e-5
    trans_mode = np.where(np.abs(eigenvalues)<tol)[0]
    qdot[trans_mode] = 0.0
# Back-transform to particle velocities
velocities = (eigenvectors @ qdot) / np.sqrt(masses)

# -- Setup ---
kin = compute_kinetic(velocities, masses)
pot = compute_potential(positions, springs)
time = np.arange(0, total_time, dt)
trajectory = []
energy = []
# -- Loop over time ---
for t in time:
    trajectory.append(positions.copy())
    energy.append((kin, pot, kin + pot))

    forces = compute_forces(nparticles, positions, springs)

    # Update positions and velocities using Velocity Verlet
    positions += velocities * dt + 0.5 * forces / masses * dt**2
    new_forces = compute_forces(nparticles, positions, springs)
    velocities += 0.5 * (forces + new_forces) / masses * dt
    # Remove center of mass motion
    if remove_com:
        vcom = np.sum(velocities * masses) / np.sum(masses)
        velocities -= vcom

    kin = compute_kinetic(velocities, masses)
    pot = compute_potential(positions, springs)

# -- end of time loop --
trajectory = np.array(trajectory)  # shape (nt, nparticles)

# --- Mass-weighted projection into eigenmodes ---
M_sqrt = np.sqrt(masses)
trajectory_massweighted = (trajectory - eq_pos) * M_sqrt[np.newaxis, :]  # (nt, nparticles)
q_traj = trajectory_massweighted @ eigenvectors              # (nt, nmodes)

# --- Identify and drop the translation mode (smallest eigenvalue) ---
mode_order = np.argsort(eigenvalues)       # ascending order
nontrivial_modes = mode_order[1:]          # drop first (translation)
eigenvalues_nt = eigenvalues[nontrivial_modes]
frequencies_nt = np.sqrt(np.abs(eigenvalues_nt))
q_traj_nt = q_traj[:, nontrivial_modes]
nmodes = len(nontrivial_modes)

In [ ]:
# @title Visualize the Simulation  { display-mode: "form" }
# --- Visualization of the dynamics ---
colors = ["red", "blue", "green"]

# --- reconstruct particle trajectories from individual modes ---
M_isqrt = 1.0 / np.sqrt(masses)
x_from_mode1 = eq_pos + (q_traj_nt[:, 0][:, None] @ eigenvectors[:, nontrivial_modes[0]][None, :]) * M_isqrt
x_from_mode2 = eq_pos + (q_traj_nt[:, 1][:, None] @ eigenvectors[:, nontrivial_modes[1]][None, :]) * M_isqrt

# --- build figure with 2 rows x 3 columns ---
fig = make_subplots(
    rows=2, cols=3,
    shared_yaxes=True,
    shared_xaxes=False,
    row_heights=[0.8, 0.2],
    vertical_spacing=0.05,
    horizontal_spacing=0.08,
    subplot_titles=(
        "Full trajectory",
        "Reconstructed: mode 1",
        "Reconstructed: mode 2",
        "", "", ""  # no titles for bottom plots
    )
)

# --- Top row: static trajectories ---
datasets = [trajectory, x_from_mode1, x_from_mode2]

for col, data in enumerate(datasets, start=1):
    for i in range(nparticles):
        fig.add_trace(
            go.Scatter(
                x=[data[0, i]], y=[time[0]],
                mode="lines", line=dict(color=colors[i]),
                name=f"Particle {i}" if col == 1 else None,  # legend only once
                showlegend=col==1),
            row=1, col=col
        )

# --- Bottom row: initial particle positions ---
for col, data in enumerate(datasets, start=1):
    init_pos = data[0]
    fig.add_trace(
        go.Scatter(
            x=init_pos, y=[0]*nparticles,
            mode="markers",
            marker=dict(size=24, color=colors),
            showlegend=col==1),
        row=2, col=col
    )

# --- Build frames ---
frames = []
for i in range(0, nsteps, frame_stride):
    pos = trajectory[i]
    pos_from_mode1 = x_from_mode1[i]
    pos_from_mode2 = x_from_mode2[i]
#    for data in [trajectory, x_from_mode1, x_from_mode2]:
    frames.append(go.Frame(data=[
        # top subplot: partial trajectories up to time i
        go.Scatter(x=trajectory[:i,0], y=time[:i], mode='lines', line=dict(color=colors[0])),
        go.Scatter(x=trajectory[:i,1], y=time[:i], mode='lines', line=dict(color=colors[1])),
        go.Scatter(x=trajectory[:i,2], y=time[:i], mode='lines', line=dict(color=colors[2])),
        go.Scatter(x=x_from_mode1[:i,0], y=time[:i], mode='lines', line=dict(color=colors[0]), showlegend=False ),
        go.Scatter(x=x_from_mode1[:i,1], y=time[:i], mode='lines', line=dict(color=colors[1]), showlegend=False ),
        go.Scatter(x=x_from_mode1[:i,2], y=time[:i], mode='lines', line=dict(color=colors[2]), showlegend=False ),
        go.Scatter(x=x_from_mode2[:i,0], y=time[:i], mode='lines', line=dict(color=colors[0]), showlegend=False ),
        go.Scatter(x=x_from_mode2[:i,1], y=time[:i], mode='lines', line=dict(color=colors[1]), showlegend=False ),
        go.Scatter(x=x_from_mode2[:i,2], y=time[:i], mode='lines', line=dict(color=colors[2]), showlegend=False ),
        # bottom subplot: moving particles
        go.Scatter(x=pos, y=[0]*nparticles, mode="markers", marker=dict(size=24, color=colors), showlegend=False ),
        go.Scatter(x=pos_from_mode1, y=[0]*nparticles, mode="markers", marker=dict(size=24, color=colors), showlegend=False ),
        go.Scatter(x=pos_from_mode2, y=[0]*nparticles, mode="markers", marker=dict(size=24, color=colors), showlegend=False ),
        ]))

delta = 0.1 * (trajectory.max() - trajectory.min())
# --- Layout ---
fig.update_layout(
    height=700,
    plot_bgcolor="white",
    # Axes: top = position vs time
    xaxis=dict(title="Position (full)", range=[trajectory.min()-delta, trajectory.max()+delta], showticklabels=False),
    xaxis2=dict(title="Position (mode 1)", range=[trajectory.min()-delta, trajectory.max()+delta], showticklabels=False),
    xaxis3=dict(title="Position (mode 2)", range=[trajectory.min()-delta, trajectory.max()+delta], showticklabels=False),
    yaxis=dict(title="Time", range=[time.min(), time.max()+dt]),
    # Bottom row axes
    xaxis4=dict(title="Position", range=[trajectory.min()-delta, trajectory.max()+delta],
                showgrid=False, zeroline=False, showline=False),
    xaxis5=dict(title="Position", range=[trajectory.min()-delta, trajectory.max()+delta],
                showgrid=False, zeroline=False, showline=False),
    xaxis6=dict(title="Position", range=[trajectory.min()-delta, trajectory.max()+delta],
                showgrid=False, zeroline=False, showline=False),
    yaxis4=dict(visible=False, range=[-1, 1]),
    yaxis5=dict(visible=False, range=[-1, 1]),
    yaxis6=dict(visible=False, range=[-1, 1]),
    updatemenus=[{
        "buttons": [
            {"args": [None, {"frame": {"duration": 50, "redraw": True},
                             "fromcurrent": True, "transition": {"duration": 0}}],
             "label": "▶ Play", "method": "animate"},
            {"args": [[None], {"frame": {"duration": 0, "redraw": False},
                               "mode": "immediate",
                               "transition": {"duration": 0}}],
             "label": "⏸ Pause", "method": "animate"}
        ],
        "direction": "left", "pad": {"r": 10, "t": 87},
        "showactive": False, "type": "buttons",
        "x": 0.5, "xanchor": "center", "y": 1.4, "yanchor": "top"
    }],
)

# --- Attach frames ---
fig.frames = frames

fig.show()

**Homework Assignment**

Pick one (or more) of the following projects:
1. Modify the code to handle a 1D systen with N particles. Find a suitable molecule that can be described by your model (e.g. acetylene).
2. Modify the code to handle one of the following 2D systems:
    * Water and ozone (how can you keep the system bent?)
    * Formaldehyde
    * Benzene (only the carbon atoms)  
3. Modify the code to handle a periodic 1D system with 2 particles

For your specific application, please address the following points:
1. Discuss which modes you expect to be able to characterize and if your model is missing any of the modes of the real system.
2. Report the modes, their frequencies and the corresponding atomic displacements. Discuss how these respect/break the symmetry of the system.
3. If you are modeling a real molecule, find in the literature (experimental or computational) some reference values for the different vibrations of the system. Compare the results of your coupled-springs model to the literature, discussing how much of the trends are reproduced and giving a tentative explanation of the agreement/disagreement of your model.  
4. How could you improve your model? Report some possible ideas. We are not asking for very long and detailed explanations, just short sentences and keywords.

NOTE: It is not necessary that the modified code produces an animation, but you should be able to visualize the results of the simulation in some way.